# Import libraries

In [1]:
%%capture
!pip install transformers sentence-transformers langchain faiss-cpu PyPDF2 -U langchain-community pypdf chromadb datasets ragas langchain_groq

In [2]:
# Import required modules
import os
import PyPDF2
from PyPDF2 import PdfReader , PdfWriter, PdfMerger
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain import LLMChain
from transformers import pipeline
from datasets import Dataset
from ragas.metrics import faithfulness, answer_relevancy
from ragas import evaluate

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAIBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAICommon has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/ragas/metrics/__init__.py:4: LangChainDeprecationWarning: As of langchain-core 0

# Upload the PDF file

In [3]:
# Load the PDF
pdf_path = 'NLP.pdf'
loader = PyPDFLoader(pdf_path)
documents = loader.load()

In [4]:
# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
chunks = text_splitter.split_documents(documents)

In [5]:
# initialize Hugging Face embeddings
embedding_model = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")
vectorstore = Chroma.from_documents(chunks, embedding_model, persist_directory='vectorstore')

<ipython-input-5-143ad68cfd9e>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
groq_api_key = ""

llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-8b-8192")

In [7]:
template = """
You are a helpful assistant. Use the context to answer the question as correctly as you can.

Context: {context}
Question: {question}
Answer:
"""
prompt = PromptTemplate(input_variables=["context", "question"], template=template)

In [8]:
model = LLMChain(llm=llm,
                 prompt=prompt,
                 verbose=True)

<ipython-input-8-578d09652ea7>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  model = LLMChain(llm=llm,


In [9]:
def query_rag(query: str):
    similarity_search_results = vectorstore.similarity_search_with_score(query, k=2)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])

    rag_response = model.invoke({"context": context_text, "question": query})
  # Get the answer from the model's output
    response_text = rag_response['text']

    # Return for evaluation
    return response_text, context_text


In [10]:
# user query input
user_query = input("Please enter your question: ")
response, context = query_rag(user_query)

Please enter your question: What is Natural Language processing


Prompt after formatting:

You are a helpful assistant. Use the context to answer the question as correctly as you can.

Context: What is NLP anyways? 
Natural Language Processing  (NLP) is deﬁned as the branch of Artiﬁcial 
Intelligence that provides computers with the capability of understanding 
text and spoken words, in the same way a human being can. 
It incorporates machine learning models, statistics, and deep learning 
models into computational linguistics i.e. rule-based modeling of human 
language to allow computers to understand text, spoken words and 
understands human language, intent, and sentiment.

What is Natural Language Understanding? 
•Natural language understanding (NLU) is a branch 
of artiﬁcial intelligence (AI) that uses computer 
software to understand input in the form of 
sentences using text or speech. 
•NLU enables computers to understand the 
sentiments expressed in a natural language used 
by humans, without the formalized syntax of 
computer languages. 
•

In [13]:
import openai

OpenAI_KEY_API=""

os.environ["OPENAI_API_KEY"] = OpenAI_KEY_API
openai.api_key = os.getenv("OPENAI_API_KEY")

# Create a dataset for evaluation
data_samples = {
    'question': [user_query],
    'answer': [response],
    'contexts': [[context]],
}
dataset = Dataset.from_dict(data_samples)

# Evaluation using faithfulness and answer relevance
score = evaluate(dataset, metrics=[faithfulness, answer_relevancy])

score_df = score.to_pandas()
print(score_df)

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[0]: RateLimitError(Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}})
ERROR:ragas.executor:Exception raised in Job[1]: RateLimitError(Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}})
ERROR:ragas.executor:Exception raised in Job[0]: RateLimitError(Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platfor

                              question  \
0  What is Natural Language processing   

                                            contexts  \
0  [What is NLP anyways? \nNatural Language Proce...   

                                              answer  faithfulness  \
0  According to the context, Natural Language Pro...           NaN   

   answer_relevancy  
0               NaN  
